<a href="https://colab.research.google.com/github/Gauravhulmukh/Chatbot-Development-for-Regional-language-using-Artificial-Intelligence/blob/master/Response_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
!pip install -U flask-cors
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin

In [3]:
# restore all of our data structures
import pickle
data = pickle.load( open( "chatbot_redsamurai_medical_training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

from google.colab import drive
drive.mount('/content/drive')

# import our chat-bot intents file
import json
with open('/content/drive/My Drive/intents.json') as json_data:
    intents = json.load(json_data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_chatbot_redsamurai_medical_logs')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.cast instead.


In [0]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [6]:
p = bow("looking for pharmacy", words)
print (p)
print (classes)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]
['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [7]:
# load our saved model
model.load('./chatbot_redsamurai_medical_model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/chatbot_redsamurai_medical_model.tflearn


In [8]:
app = Flask(__name__)
CORS(app)

@app.route("/redsam/api/v1.0/classify", methods=['POST'])
def classify():
    ERROR_THRESHOLD = 0.25
    
    sentence = request.json['sentence']
    
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    response = jsonify(return_list)
    return response

def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

# running REST interface
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


In [9]:
classify_local('Hi')

[('greeting', '0.734897')]

In [10]:
classify_local('what help you offer?')

[('options', '0.9250798')]

In [11]:
classify_local('What about adverse reaction?')

[('adverse_drug', '0.4292215'), ('goodbye', '0.37230432')]

In [12]:
classify_local('Checking blood pressure results for patient')

[('blood_pressure_search', '0.82664067')]

In [13]:
classify_local('please load patient blood pressure module')

[('blood_pressure_search', '0.81243235')]

In [14]:
classify_local('Any recommendations for adverse drugs?')

[('adverse_drug', '0.68265617'), ('hospital_search', '0.2506993')]

In [15]:
classify_local('lookup pharmacy by code pls')

[('pharmacy_search', '0.65254104')]

In [16]:
classify_local('bye for now')

[('goodbye', '0.6477864')]